In [2]:
import pandas as pd
import numpy as np

XTrain = pd.read_csv('c:/Users/I051796/Projects/CES/data/input_train.csv', sep=';')
XTest = pd.read_csv('c:/Users/I051796/Projects/CES/data/input_test.csv', sep=';')
YTrain = pd.read_csv('c:/Users/I051796/Projects/CES/data/label.csv', sep=';')

In [3]:
import nltk
import re
import string
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

nltk.download('stopwords')
    
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("french"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"\?\?", "?", text)
    
    text = text.split()
    stemmer = SnowballStemmer('french')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jacques\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
XTrain['cleanedQuestion'] = XTrain['question'].map(lambda x: clean_text(x))

In [5]:
pd.set_option("display.max_colwidth",10000)
#XTrain["cleanedQuestion"].to_csv("c:/temp/clean.txt")

In [6]:
# TF IDF
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

vectorizer = TfidfVectorizer(strip_accents='ascii', stop_words=stopwords.words('french'))
XTFIDFVectorizedTrain = vectorizer.fit_transform(XTrain['cleanedQuestion'])
XTFIDFVectorizedTrain = pd.DataFrame(XTFIDFVectorizedTrain.toarray())
XTFIDFVectorizedTrain.head()

,0,1,2,3,4,5,6,7,8,9,...,8324,8325,8326,8327,8328,8329,8330,8331,8332,8333
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

kfold = KFold(n_splits=2, random_state=1971)
gbm = xgb.XGBClassifier(max_depth=4, n_estimators=20, n_jobs=3, tree_method='gpu_hist', n_gpus=1)
results = cross_val_score(gbm, XTFIDFVectorizedTrain, YTrain.intention, cv=kfold)

d:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [8]:
results

array([0.47907324, 0.47757848])

In [9]:
gbm.fit(XTFIDFVectorizedTrain, YTrain.intention)
YPredicted = gbm.predict(XTFIDFVectorizedTrain)

XGBoostError: b'[22:35:33] D:/git/xgboost/src/tree/updater_gpu_hist.cu:785: Exception in gpu_hist: bad allocation\n'

In [ ]:
import matplotlib.pyplot as plt
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')

    plt.xlabel('Predicted label')

In [ ]:
%matplotlib inline

import sklearn
from sklearn.metrics import confusion_matrix

cnf_matrix = confusion_matrix(YTrue, YPredicted)

print(sklearn.metrics.classification_report(YTrue, YPredicted))

plt.figure(figsize=(20,20))

plot_confusion_matrix(cnf_matrix, normalize=False, classes = np.unique(YTrain['intention']))

plt.show()


In [ ]:
# 28 (mode) is confusing predominantly with 31 and 22
from sklearn.manifold import TSNE
import numpy as np
from sklearn.decomposition import IncrementalPCA

XTFIDFVectorizedTrain['ID'] = XTFIDFVectorizedTrain.index
XYTFIDFVectorizedTrain = XTFIDFVectorizedTrain.merge(YTrain, on='ID')
XYTFIDFVectorizedTrain = XYTFIDFVectorizedTrain[XYTFIDFVectorizedTrain['intention'].isin([28,22,31])]
XYTFIDFVectorizedTrain = XYTFIDFVectorizedTrain.drop(['ID', 'intention'], axis=1)
XYTFIDFVectorizedTrain.shape

In [ ]:

#tsne = TSNE(n_components=2)
#TSNETrain = tsne.fit_transform(XTFIDFVectorizedTrain)
#tsne = TSNE(n_components=2)

ipca = IncrementalPCA(n_components=2, batch_size=3)
PCATrain = pd.DataFrame(ipca.fit_transform(XYTFIDFVectorizedTrain))
PCATrain.shape

In [ ]:
import matplotlib.cm as cm

plt.figure(figsize=(15,15))
filterYTrain = YTrain[YTrain['intention'].isin([28,22,31])]
plt.scatter(PCATrain[0], PCATrain[1], c=filterYTrain['intention'], cmap=cm.seismic_r)
plt.show()

In [ ]:
YXTrain = XTrain.merge(YTrain, on='ID')

In [ ]:
pd.set_option("display.max_colwidth",10000)
## 22 is a fuzzy question on the drug
YXTrain[YXTrain['intention']==22].head(5)

In [ ]:
YXTrain[YXTrain['intention']==31].head(50)